In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os

base_path = "/content/drive/MyDrive/ds_Raj_Kolhe"  # your correct folder
csv_folder = os.path.join(base_path, "csv_files")

print("Project base path:", base_path)
print("CSV folder path:  ", csv_folder)
print("\nFiles inside csv_files:")

for f in sorted(os.listdir(csv_folder)):
    print(" -", f)


Project base path: /content/drive/MyDrive/ds_Raj_Kolhe
CSV folder path:   /content/drive/MyDrive/ds_Raj_Kolhe/csv_files

Files inside csv_files:
 - fear_greed_index.csv
 - historical_data.csv


In [4]:
import pandas as pd
import os

# Use your real filenames
trade_csv = os.path.join(csv_folder, "historical_data.csv")
fg_csv    = os.path.join(csv_folder, "fear_greed_index.csv")

print("Reading files:\n", trade_csv, "\n", fg_csv)

# Load datasets
trades = pd.read_csv(trade_csv, low_memory=False)
fear_greed = pd.read_csv(fg_csv, low_memory=False)

print("Loaded successfully!")


Reading files:
 /content/drive/MyDrive/ds_Raj_Kolhe/csv_files/historical_data.csv 
 /content/drive/MyDrive/ds_Raj_Kolhe/csv_files/fear_greed_index.csv
Loaded successfully!


In [5]:
print("=== TRADES DATA ===")
print("Shape:", trades.shape)
display(trades.head())
display(trades.dtypes)
print("Missing values:")
print(trades.isna().sum().sort_values(ascending=False).head(10))

print("\n=== FEAR & GREED DATA ===")
print("Shape:", fear_greed.shape)
display(fear_greed.head())
display(fear_greed.dtypes)
print("Missing values:")
print(fear_greed.isna().sum())


=== TRADES DATA ===
Shape: (211224, 16)


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


,0
Account,object
Coin,object
Execution Price,float64
Size Tokens,float64
Size USD,float64
Side,object
Timestamp IST,object
Start Position,float64
Direction,object
Closed PnL,float64


Missing values:
Account            0
Coin               0
Execution Price    0
Size Tokens        0
Size USD           0
Side               0
Timestamp IST      0
Start Position     0
Direction          0
Closed PnL         0
dtype: int64

=== FEAR & GREED DATA ===
Shape: (2644, 4)


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


,0
timestamp,int64
value,int64
classification,object
date,object


Missing values:
timestamp         0
value             0
classification    0
date              0
dtype: int64


In [6]:
# STEP 10: Standardize column names

def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("__", "_")
    )
    return df

trades = clean_columns(trades)
fear_greed = clean_columns(fear_greed)

print("Trades columns:", trades.columns.tolist())
print("\nFear & Greed columns:", fear_greed.columns.tolist())


Trades columns: ['account', 'coin', 'execution_price', 'size_tokens', 'size_usd', 'side', 'timestamp_ist', 'start_position', 'direction', 'closed_pnl', 'transaction_hash', 'order_id', 'crossed', 'fee', 'trade_id', 'timestamp']

Fear & Greed columns: ['timestamp', 'value', 'classification', 'date']


In [7]:
# STEP 11: Parse datetime columns

# 1. Convert trades['timestamp_ist']
trades['timestamp_ist'] = pd.to_datetime(
    trades['timestamp_ist'],
    format="%d-%m-%Y %H:%M",
    errors="coerce"
)

print("Trades timestamp_ist parsed. Missing values:", trades['timestamp_ist'].isna().sum())

# 2. Convert fear_greed['date']
fear_greed['date'] = pd.to_datetime(fear_greed['date'], errors='coerce')

# 3. Convert fear_greed['timestamp'] (epoch -> datetime)
fear_greed['timestamp_converted'] = pd.to_datetime(
    fear_greed['timestamp'], unit='s', errors='coerce'
)

print("Fear & Greed timestamps converted.")
fear_greed[['timestamp', 'timestamp_converted', 'date']].head()


Trades timestamp_ist parsed. Missing values: 0
Fear & Greed timestamps converted.


,timestamp,timestamp_converted,date
0,1517463000,2018-02-01 05:30:00,2018-02-01
1,1517549400,2018-02-02 05:30:00,2018-02-02
2,1517635800,2018-02-03 05:30:00,2018-02-03
3,1517722200,2018-02-04 05:30:00,2018-02-04
4,1517808600,2018-02-05 05:30:00,2018-02-05


In [8]:
# STEP 12: Add new datetime features

# For trades
trades['trade_date'] = trades['timestamp_ist'].dt.date
trades['trade_hour'] = trades['timestamp_ist'].dt.hour
trades['trade_day_of_week'] = trades['timestamp_ist'].dt.day_name()

# For fear_greed
fear_greed['fg_date'] = fear_greed['date'].dt.date
fear_greed['fg_day_of_week'] = fear_greed['date'].dt.day_name()

print("New datetime features added!")
trades[['timestamp_ist', 'trade_date', 'trade_hour', 'trade_day_of_week']].head()


New datetime features added!


,timestamp_ist,trade_date,trade_hour,trade_day_of_week
0,2024-12-02 22:50:00,2024-12-02,22,Monday
1,2024-12-02 22:50:00,2024-12-02,22,Monday
2,2024-12-02 22:50:00,2024-12-02,22,Monday
3,2024-12-02 22:50:00,2024-12-02,22,Monday
4,2024-12-02 22:50:00,2024-12-02,22,Monday


In [9]:
# STEP 13: Merge datasets by date
import pandas as pd
import os

# ensure trade_date and fg_date exist (from previous steps)
print("Example trade_date:", trades['trade_date'].head(3))
print("Example fg_date:", fear_greed['fg_date'].head(3))

# Reduce fear_greed to one row per date (if duplicates exist, take mean value)
fg_daily = (
    fear_greed
    .groupby('fg_date', as_index=False)
    .agg({
        'value': 'mean',               # numeric FGI value (0-100)
        'classification': lambda x: x.mode().iat[0] if len(x)>0 else None
    })
    .rename(columns={'value':'fgi_value','classification':'fgi_class'})
)

print("FG daily sample:")
display(fg_daily.head())

# Merge trades with fg_daily on date
trades['trade_date'] = pd.to_datetime(trades['trade_date'])
fg_daily['fg_date'] = pd.to_datetime(fg_daily['fg_date'])

# left join so every trade keeps its row; missing FGI will be NaN
trades_merged = trades.merge(
    fg_daily,
    how='left',
    left_on='trade_date',
    right_on='fg_date'
)

print("Merged dataset shape:", trades_merged.shape)
display(trades_merged[['trade_date','fgi_value','fgi_class']].head())


Example trade_date: 0    2024-12-02
1    2024-12-02
2    2024-12-02
Name: trade_date, dtype: object
Example fg_date: 0    2018-02-01
1    2018-02-02
2    2018-02-03
Name: fg_date, dtype: object
FG daily sample:


,fg_date,fgi_value,fgi_class
0,2018-02-01,30.0,Fear
1,2018-02-02,15.0,Extreme Fear
2,2018-02-03,40.0,Fear
3,2018-02-04,24.0,Extreme Fear
4,2018-02-05,11.0,Extreme Fear


Merged dataset shape: (211224, 22)


,trade_date,fgi_value,fgi_class
0,2024-12-02,80.0,Extreme Greed
1,2024-12-02,80.0,Extreme Greed
2,2024-12-02,80.0,Extreme Greed
3,2024-12-02,80.0,Extreme Greed
4,2024-12-02,80.0,Extreme Greed


In [10]:
# STEP 14: Create sentiment buckets
def sentiment_bucket(val):
    if pd.isna(val):
        return "unknown"
    val = float(val)
    if val <= 10:
        return "extreme_fear"
    if val <= 25:
        return "fear"
    if val <= 40:
        return "neutral_fear"
    if val <= 60:
        return "neutral"
    if val <= 75:
        return "greed"
    return "extreme_greed"

trades_merged['fgi_bucket'] = trades_merged['fgi_value'].apply(sentiment_bucket)
trades_merged['fgi_bucket'] = trades_merged['fgi_bucket'].astype('category')

# Quick counts
print("Counts per sentiment bucket:")
display(trades_merged['fgi_bucket'].value_counts(dropna=False))


Counts per sentiment bucket:


,count
fgi_bucket,
neutral,58035
greed,48197
neutral_fear,41201
extreme_greed,32421
fear,30474
extreme_fear,890
unknown,6


In [11]:
# Fixed account-level aggregation (safe and robust)
import pandas as pd

# Ensure closed_pnl numeric
trades_merged['closed_pnl'] = pd.to_numeric(trades_merged['closed_pnl'], errors='coerce')

# Try the simple grouped agg with observed=True first (fast)
try:
    account_summary = (
        trades_merged
        .groupby(['fgi_bucket','account'], observed=True, as_index=False)
        .agg(
            trades=('closed_pnl','count'),
            avg_pnl=('closed_pnl','mean'),
            total_pnl=('closed_pnl','sum')
        )
    )
    print("Aggregated with groupby(...).agg() successfully.")
except Exception as e:
    print("groupby.agg failed with error:", e)
    print("Falling back to groupby.apply (slower but robust).")
    # Fallback: apply a function per group (this always works)
    def _agg_grp(g):
        return pd.Series({
            'trades': g['closed_pnl'].count(),
            'avg_pnl': g['closed_pnl'].mean(),
            'total_pnl': g['closed_pnl'].sum()
        })
    account_summary = trades_merged.groupby(['fgi_bucket','account'], observed=True).apply(_agg_grp).reset_index()

# Show results
print("Account summary rows:", len(account_summary))
display(account_summary.head(10))


Aggregated with groupby(...).agg() successfully.
Account summary rows: 167


,fgi_bucket,account,trades,avg_pnl,total_pnl
0,extreme_fear,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,30,2.397327,71.919819
1,extreme_fear,0x47add9a56df66b524d5e2c1993a43cde53b6ed85,18,-10.880081,-195.841458
2,extreme_fear,0x4acb90e786d897ecffb614dc822eb231b4ffb9f4,12,41.943286,503.319430
3,extreme_fear,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,13,655.212731,8517.765500
4,extreme_fear,0x8477e447846c758f5a675856001ea72298fd9cb5,1,0.000000,0.000000
5,extreme_fear,0xa0feb3725a9335f49874d7cd8eaad6be45b27416,13,0.000000,0.000000
6,extreme_fear,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,10,0.000000,0.000000
7,extreme_fear,0xbd5fead7180a9c139fa51a103cb6a2ce86ddb5c3,90,-536.767975,-48309.117760
8,extreme_fear,0xbee1707d6b44d4d52bfe19e41f8a828645437aab,703,4.106383,2886.786952
9,extreme_greed,0x083384f897ee0f19899168e3b1bec365f52a9012,945,-42.626810,-40282.335904


In [14]:
# STEP 16: Save summaries to outputs
import pandas as pd # Ensure pandas is imported

out_folder = "/content/drive/MyDrive/ds_Raj_Kolhe/outputs"

# Create trade_summary, which was missing
trade_summary = (
    trades_merged
    .groupby('fgi_bucket', observed=True, as_index=False)
    .agg(
        trades_count=('closed_pnl','count'),
        mean_pnl=('closed_pnl','mean'),
        total_pnl=('closed_pnl','sum')
    )
)

account_summary_path = out_folder + "/account_summary.csv"
trade_summary_path   = out_folder + "/trade_summary.csv"

account_summary.to_csv(account_summary_path, index=False)
trade_summary.to_csv(trade_summary_path, index=False)

print("Saved:")
print(" -", account_summary_path)
print(" -", trade_summary_path)


Saved:
 - /content/drive/MyDrive/ds_Raj_Kolhe/outputs/account_summary.csv
 - /content/drive/MyDrive/ds_Raj_Kolhe/outputs/trade_summary.csv


In [15]:
# STEP 17: Plots and save figures
import matplotlib.pyplot as plt

# 1) Mean PnL by bucket bar chart
mean_by_bucket = trades_merged.groupby('fgi_bucket')['closed_pnl'].mean().reindex(trades_merged['fgi_bucket'].cat.categories)
plt.figure(figsize=(8,4))
ax = mean_by_bucket.plot(kind='bar')
ax.set_title('Mean Closed PnL by FGI Bucket')
ax.set_ylabel('Mean Closed PnL')
ax.set_xlabel('FGI Bucket')
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
fig1 = ax.get_figure()
fig1_path = out_folder + "/mean_pnl_by_bucket.png"
fig1.savefig(fig1_path)
plt.close(fig1)

# 2) Boxplot of Closed PnL by bucket (sample to speed)
sampled = trades_merged[trades_merged['closed_pnl'].notna()].sample(n=min(20000,len(trades_merged)), random_state=42)
plt.figure(figsize=(10,5))
bp = sampled.boxplot(column='closed_pnl', by='fgi_bucket', showfliers=False)
plt.title('Closed PnL distribution by FGI Bucket (no outliers shown)')
plt.suptitle('')
plt.xlabel('FGI Bucket')
plt.ylabel('Closed PnL')
plt.xticks(rotation=45)
plt.tight_layout()
fig2 = plt.gcf()
fig2_path = out_folder + "/boxplot_pnl_by_bucket.png"
fig2.savefig(fig2_path)
plt.close(fig2)

print("Saved figures:")
print(" -", fig1_path)
print(" -", fig2_path)


/tmp/ipython-input-1694472880.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_by_bucket = trades_merged.groupby('fgi_bucket')['closed_pnl'].mean().reindex(trades_merged['fgi_bucket'].cat.categories)


Saved figures:
 - /content/drive/MyDrive/ds_Raj_Kolhe/outputs/mean_pnl_by_bucket.png
 - /content/drive/MyDrive/ds_Raj_Kolhe/outputs/boxplot_pnl_by_bucket.png


<Figure size 1000x500 with 0 Axes>

In [16]:
# STEP 18: Kruskal-Wallis + Dunn posthoc if significant
from scipy.stats import kruskal
import numpy as np

groups = []
labels = []
for b in trades_merged['fgi_bucket'].cat.categories:
    arr = trades_merged.loc[trades_merged['fgi_bucket']==b, 'closed_pnl'].dropna()
    if len(arr) >= 10:
        groups.append(arr.values)
        labels.append(b)

if len(groups) >= 2:
    stat, p = kruskal(*groups)
    print("Kruskal-Wallis H-statistic:", stat, "p-value:", p)
    if p < 0.05:
        print("Significant difference (p < 0.05). Running pairwise Dunn post-hoc...")
        # Dunn's test (via scikit-posthocs). If not installed, install it first.
        try:
            import scikit_posthocs as sp
        except Exception:
            !pip install scikit-posthocs -q
            import scikit_posthocs as sp
        # prepare DataFrame for sp.posthoc_dunn
        df_dunn = trades_merged[['fgi_bucket','closed_pnl']].dropna()
        dunn_res = sp.posthoc_dunn(df_dunn, val_col='closed_pnl', group_col='fgi_bucket', p_adjust='bonferroni')
        print("Dunn post-hoc (p-values matrix):")
        display(dunn_res)
        # save
        dunn_res.to_csv(out_folder + "/dunn_posthoc_pvals.csv")
        print("Saved Dunn p-values to:", out_folder + "/dunn_posthoc_pvals.csv")
    else:
        print("No statistically significant differences detected (p >= 0.05).")
else:
    print("Not enough groups with >=10 observations. Labels considered:", labels)


Kruskal-Wallis H-statistic: 1318.1329274578518 p-value: 7.530386564229788e-283
Significant difference (p < 0.05). Running pairwise Dunn post-hoc...
Dunn post-hoc (p-values matrix):


,extreme_fear,extreme_greed,fear,greed,neutral,neutral_fear,unknown
extreme_fear,1.000000e+00,7.850312e-46,1.750884e-13,6.367409e-24,2.120718e-25,2.327083e-42,0.000003
extreme_greed,7.850312e-46,1.000000e+00,2.783397e-175,5.886030e-82,1.784963e-75,1.231024e-01,0.000832
fear,1.750884e-13,2.783397e-175,1.000000e+00,2.093624e-31,7.364932e-42,9.535978e-162,0.000065
greed,6.367409e-24,5.886030e-82,2.093624e-31,1.000000e+00,1.000000e+00,2.208479e-68,0.000180
neutral,2.120718e-25,1.784963e-75,7.364932e-42,1.000000e+00,1.000000e+00,6.351564e-62,0.000203
neutral_fear,2.327083e-42,1.231024e-01,9.535978e-162,2.208479e-68,6.351564e-62,1.000000e+00,0.000669
unknown,2.532272e-06,8.319331e-04,6.532341e-05,1.803529e-04,2.026482e-04,6.686334e-04,1.000000


Saved Dunn p-values to: /content/drive/MyDrive/ds_Raj_Kolhe/outputs/dunn_posthoc_pvals.csv


In [17]:
# STEP 19: Save merged cleaned dataset sample (full might be large)
merged_path = out_folder + "/trades_merged_sample.csv"
# Save a sample (e.g., first 200k rows or full if small)
trades_merged.head(200000).to_csv(merged_path, index=False)
print("Saved merged sample to:", merged_path)


Saved merged sample to: /content/drive/MyDrive/ds_Raj_Kolhe/outputs/trades_merged_sample.csv


In [18]:
# STEP 20: Auto-generate 6 starter insight bullets
top_buckets = trade_summary.sort_values('mean_pnl', ascending=False).head(3)['fgi_bucket'].tolist()
bottom_buckets = trade_summary.sort_values('mean_pnl', ascending=True).head(3)['fgi_bucket'].tolist()

insights = [
    f"1) Most trades: {trade_summary.sort_values('trades_count', ascending=False).iloc[0]['fgi_bucket']} ({int(trade_summary.sort_values('trades_count', ascending=False).iloc[0]['trades_count'])} trades).",
    f"2) Highest average closed PnL buckets: {', '.join(map(str, top_buckets))}.",
    f"3) Lowest average closed PnL buckets: {', '.join(map(str, bottom_buckets))}.",
    f"4) Overall win rate across all trades: {trades_merged['closed_pnl'].gt(0).sum()}/{trades_merged['closed_pnl'].notna().sum()} = {trades_merged['closed_pnl'].gt(0).sum()/trades_merged['closed_pnl'].notna().sum():.2%}.",
    "5) Consider investigating trading strategies concentrated under 'extreme_fear' (low mean_pnl and win_rate) to understand risk exposure.",
    "6) If Kruskal-Wallis test is significant, pairwise Dunn results (dunn_posthoc_pvals.csv) identify which sentiment buckets differ significantly."
]

print("Starter insights:")
for s in insights:
    print(s)
# Also save to a text file for README
with open(out_folder + "/starter_insights.txt", "w") as f:
    f.write("\n".join(insights))
print("Saved starter insights to:", out_folder + "/starter_insights.txt")


Starter insights:
1) Most trades: neutral (58035 trades).
2) Highest average closed PnL buckets: unknown, extreme_greed, neutral.
3) Lowest average closed PnL buckets: extreme_fear, fear, greed.
4) Overall win rate across all trades: 86869/211224 = 41.13%.
5) Consider investigating trading strategies concentrated under 'extreme_fear' (low mean_pnl and win_rate) to understand risk exposure.
6) If Kruskal-Wallis test is significant, pairwise Dunn results (dunn_posthoc_pvals.csv) identify which sentiment buckets differ significantly.
Saved starter insights to: /content/drive/MyDrive/ds_Raj_Kolhe/outputs/starter_insights.txt


In [19]:
# Create a smaller sample under 100MB
small_sample = trades_merged.sample(n=50000, random_state=42)
small_sample_path = "/content/drive/MyDrive/ds_Raj_Kolhe/outputs/trades_merged_sample_small.csv"

small_sample.to_csv(small_sample_path, index=False)

print("Saved smaller sample at:", small_sample_path)


Saved smaller sample at: /content/drive/MyDrive/ds_Raj_Kolhe/outputs/trades_merged_sample_small.csv
